In [5]:
from vnstock import Vnstock
from vnstock import Listing
import talib as ta 
import mplfinance as mpf
import pandas as pd
import numpy as np


In [6]:
def find_low_index(series, current_index, lookback = 20):
    if current_index < lookback:
        return None
    window = series.iloc[current_index-lookback:current_index]
    if window.empty or window.isnull().all():
        return None
    return window.idxmin()

def find_high_index(series, current_index, lookback = 20):
    if current_index < lookback:
        return None
    window = series.iloc[current_index-lookback:current_index]
    if window.empty or window.isnull().all():
        return None
    return window.idxmax()

In [7]:
def test(symbol = 'AAA', start = '2018-01-01', end = '2025-01-01'):
    # print(f"CCI strategy for {symbol} from {start} to {end}")
    if symbol is not None:
        try:
            stock = Vnstock().stock(symbol)
            df = stock.quote.history(start = start, end = end, interval = '1D')
            

            df['time'] = pd.DatetimeIndex(df['time'])
            df.set_index('time', inplace=True)

            cci = ta.CCI(df['high'], df['low'], df['close'], timeperiod=14)
            macd, macdsignal, macdhist = ta.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
            rsi = ta.RSI(df['close'], timeperiod=14)

            PnL = []
            stocks_bought = 0
            buy_price = 0

            for i in range(len(df)):
                rsi_bearish = False
                rsi_bullish = False
                macd_bearish = False
                macd_bullish = False
                
                
                prev_low_idx = find_low_index(df['low'], i, 20)
                prev_high_idx = find_high_index(df['high'], i, 20)
                # print(f"{prev_low_idx}, {prev_high_idx}")
                prev_low_rsi_idx = find_low_index(rsi, i, 20)
                prev_high_rsi_idx = find_high_index(rsi, i, 20)   

                prev_low_macd_idx = find_low_index(macd, i, 26)
                prev_high_macd_idx = find_high_index(macd, i, 26)
    

                # Bullish Divergence
                if (prev_low_idx is not None) and (prev_low_rsi_idx is not None) and (prev_low_macd_idx is not None):
                    try:
                        prev_low_pos = prev_low_idx if isinstance(prev_low_idx, int) else df.index.get_loc(prev_low_idx)
                        prev_low_rsi_pos = prev_low_rsi_idx if isinstance(prev_low_rsi_idx, int) else df.index.get_loc(prev_low_rsi_idx)
                        prev_low_macd_pos = prev_low_macd_idx if isinstance(prev_low_macd_idx, int) else df.index.get_loc(prev_low_macd_idx)
                        if df['low'].iloc[i] < df['low'].iloc[prev_low_pos]:
                            if rsi.iloc[i] > rsi.iloc[prev_low_rsi_pos]:
                                rsi_bullish = True
                            if macd.iloc[i] > macd.iloc[prev_low_macd_pos]:  
                                macd_bullish = True
                    except (TypeError, KeyError) as e:
                        print(f"Error in bullish divergence at index {i}: {e}")

                # Bearish Divergence
                if (prev_high_idx is not None) and (prev_high_rsi_idx is not None) and (prev_high_macd_idx is not None):
                    try:
                        prev_high_pos = prev_high_idx if isinstance(prev_high_idx, int) else df.index.get_loc(prev_high_idx)
                        prev_high_rsi_pos = prev_high_rsi_idx if isinstance(prev_high_rsi_idx, int) else df.index.get_loc(prev_high_rsi_idx)
                        prev_high_macd_pos = prev_high_macd_idx if isinstance(prev_high_macd_idx, int) else df.index.get_loc(prev_high_macd_idx)
                        if df['high'].iloc[i] > df['high'].iloc[prev_high_pos]:
                            if rsi.iloc[i] < rsi.iloc[prev_high_rsi_pos]:
                                rsi_bearish = True
                            if macd.iloc[i] < macd.iloc[prev_high_macd_pos]: 
                                macd_bearish = True
                    except (TypeError, KeyError) as e:
                        print(f"Error in bearish divergence at index {i}: {e}")
                
                
                if ((cci.iloc[i-1] < -100 and cci.iloc[i] > -100) 
                    or (rsi.iloc[i-1] < 30 and rsi.iloc[i] > 30) 
                    or (macdhist.iloc[i-1] < 0 and macdhist.iloc[i] > 0 and (rsi.iloc[i-1] < 45 or rsi.iloc[i-1] > 55))
                    or rsi_bullish == True or macd_bullish == True):
                    stocks_bought += 1
                    buy_price += df['close'].iloc[i]
                    # print(f"Mua 1 phiếu giá {buy_price}")
                elif (((cci.iloc[i-1] > 100 and cci.iloc[i] < 100)
                      or (rsi.iloc[i-1] > 70 and rsi.iloc[i] < 70)
                       or (macdhist.iloc[i-1] > 0 and macdhist.iloc[i] < 0 and (rsi.iloc[i-1] < 45 or rsi.iloc[i-1] > 55))
                       or rsi_bearish or macd_bearish)
                         and stocks_bought > 0):
                    
                    sell_price = df['close'].iloc[i]*stocks_bought
                    PnL.append((sell_price - buy_price)*100/buy_price)
                    # print(f"Bán {stocks_bought} tổng giá {sell_price}")
                    buy_price = 0
                    stocks_bought = 0

            return pd.Series(PnL)
        
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None
    else:
        return None


In [8]:
results = test()
# print(results)
print("Mã AAA - từ 2018-01-01 đến 2025-01-01")
profit = results[results > 0] 
print(f"Tổng số lần mua bán:    {len(results)}")
print(f"Số lần có lãi:          {len(profit)}")
print(f"Trung bình lãi:         {profit.mean():.2f}%")
loss = results[results < 0]
print(f"Số lần lỗ:              {len(loss)}")
print(f"Trung bình lỗ:          {loss.mean():.2f}%")


Mã AAA - từ 2018-01-01 đến 2025-01-01
Tổng số lần mua bán:    57
Số lần có lãi:          34
Trung bình lãi:         4.94%
Số lần lỗ:              22
Trung bình lỗ:          -3.28%


In [9]:
listing = Listing()


Thử với nhiều mã chứng khoán hơn

In [10]:
# results = pd.DataFrame(columns=['symbol', 'profit'])

# Test random symbols
symbols = listing.all_symbols()
symbols = symbols['symbol'] 
# symbols = symbols.sample(100).tolist()

start = '2018-01-01'
end = '2025-01-01'

total_result = pd.Series()
total_profit = pd.Series()
total_breakeven = pd.Series()
total_loss = pd.Series()

for symbol in symbols:
    print(f"\nMã {symbol}: ", end="")
    results = test(symbol, start, end)
    if results is None:
        print(f"Không có dữ liệu cho mã {symbol}.")
        continue

    total_result = pd.concat([total_result, results], ignore_index=True)

    print(f"{len(results)} lần mua bán, ", end="")

    profit = results[results > 0] 
    total_profit = pd.concat([total_profit, profit], ignore_index=True)
    print(f"{len(profit)} lần lãi, trung bình {profit.mean():.2f}%, ", end="")
    
    breakeven = results[results == 0]
    total_breakeven = pd.concat([total_breakeven, breakeven], ignore_index=True)
    print(f"{len(breakeven)} lần hòa vốn, ", end="")

    loss = results[results < 0]
    total_loss = pd.concat([total_loss, loss], ignore_index=True)
    print(f"{len(loss)} lần lỗ, trung bình {loss.mean():.2f}%")


Mã YTC: 

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


44 lần mua bán, 26 lần lãi, trung bình 14.82%, 5 lần hòa vốn, 13 lần lỗ, trung bình -8.97%

Mã YEG: 46 lần mua bán, 28 lần lãi, trung bình 7.87%, 0 lần hòa vốn, 18 lần lỗ, trung bình -8.77%

Mã YBM: 61 lần mua bán, 32 lần lãi, trung bình 6.46%, 2 lần hòa vốn, 27 lần lỗ, trung bình -3.38%

Mã YBC: 39 lần mua bán, 21 lần lãi, trung bình 25.20%, 4 lần hòa vốn, 14 lần lỗ, trung bình -6.91%

Mã XPH: 68 lần mua bán, 36 lần lãi, trung bình 11.17%, 5 lần hòa vốn, 27 lần lỗ, trung bình -5.81%

Mã XMP: 31 lần mua bán, 19 lần lãi, trung bình 7.18%, 4 lần hòa vốn, 8 lần lỗ, trung bình -5.52%

Mã XMD: 29 lần mua bán, 15 lần lãi, trung bình 102.98%, 1 lần hòa vốn, 13 lần lỗ, trung bình -4.68%

Mã XMC: 42 lần mua bán, 26 lần lãi, trung bình 5.61%, 0 lần hòa vốn, 16 lần lỗ, trung bình -5.64%

Mã XLV: 30 lần mua bán, 20 lần lãi, trung bình 5.98%, 3 lần hòa vốn, 7 lần lỗ, trung bình -7.27%

Mã XHC: 53 lần mua bán, 30 lần lãi, trung bình 8.06%, 10 lần hòa vốn, 13 lần lỗ, trung bình -6.87%

Mã XDH: 26 lần

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


0 lần mua bán, 0 lần lãi, trung bình nan%, 0 lần hòa vốn, 0 lần lỗ, trung bình nan%

Mã X77: 

...

0 lần mua bán, 0 lần lãi, trung bình nan%, 0 lần hòa vốn, 0 lần lỗ, trung bình nan%

Mã BCR: 7 lần mua bán, 3 lần lãi, trung bình 6.37%, 0 lần hòa vốn, 4 lần lỗ, trung bình -8.54%

Mã BCP: 66 lần mua bán, 37 lần lãi, trung bình 10.55%, 8 lần hòa vốn, 21 lần lỗ, trung bình -6.20%

Mã BCO: 

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


0 lần mua bán, 0 lần lãi, trung bình nan%, 0 lần hòa vốn, 0 lần lỗ, trung bình nan%

Mã BCM: 59 lần mua bán, 29 lần lãi, trung bình 7.62%, 0 lần hòa vốn, 30 lần lỗ, trung bình -2.87%

Mã BCG: 60 lần mua bán, 32 lần lãi, trung bình 7.03%, 1 lần hòa vốn, 27 lần lỗ, trung bình -3.64%

Mã BCF: 43 lần mua bán, 23 lần lãi, trung bình 3.19%, 4 lần hòa vốn, 16 lần lỗ, trung bình -2.70%

Mã BCE: 58 lần mua bán, 37 lần lãi, trung bình 3.88%, 1 lần hòa vốn, 20 lần lỗ, trung bình -2.98%

Mã BCC: 65 lần mua bán, 42 lần lãi, trung bình 4.36%, 3 lần hòa vốn, 20 lần lỗ, trung bình -4.05%

Mã BCB: 2 lần mua bán, 1 lần lãi, trung bình 17.65%, 1 lần hòa vốn, 0 lần lỗ, trung bình nan%

Mã BCA: 27 lần mua bán, 11 lần lãi, trung bình 9.05%, 1 lần hòa vốn, 15 lần lỗ, trung bình -5.36%

Mã BBT: 66 lần mua bán, 34 lần lãi, trung bình 8.94%, 5 lần hòa vốn, 27 lần lỗ, trung bình -6.23%

Mã BBS: 49 lần mua bán, 33 lần lãi, trung bình 10.43%, 3 lần hòa vốn, 13 lần lỗ, trung bình -4.92%

Mã BBM: 40 lần mua bán, 25 

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


0 lần mua bán, 0 lần lãi, trung bình nan%, 0 lần hòa vốn, 0 lần lỗ, trung bình nan%

Mã AIC: 37 lần mua bán, 21 lần lãi, trung bình 5.81%, 2 lần hòa vốn, 14 lần lỗ, trung bình -4.30%

Mã AGX: 49 lần mua bán, 27 lần lãi, trung bình 10.25%, 10 lần hòa vốn, 12 lần lỗ, trung bình -6.38%

Mã AGR: 65 lần mua bán, 44 lần lãi, trung bình 5.54%, 1 lần hòa vốn, 20 lần lỗ, trung bình -3.33%

Mã AGP: 62 lần mua bán, 32 lần lãi, trung bình 6.94%, 4 lần hòa vốn, 26 lần lỗ, trung bình -3.34%

Mã AGM: 51 lần mua bán, 36 lần lãi, trung bình 8.95%, 2 lần hòa vốn, 13 lần lỗ, trung bình -7.59%

Mã AGG: 49 lần mua bán, 26 lần lãi, trung bình 4.21%, 1 lần hòa vốn, 22 lần lỗ, trung bình -2.58%

Mã AGF: 55 lần mua bán, 31 lần lãi, trung bình 8.91%, 3 lần hòa vốn, 21 lần lỗ, trung bình -7.47%

Mã AGE: 7 lần mua bán, 2 lần lãi, trung bình 9.15%, 1 lần hòa vốn, 4 lần lỗ, trung bình -10.87%

Mã AG1: 51 lần mua bán, 21 lần lãi, trung bình 7.11%, 7 lần hòa vốn, 23 lần lỗ, trung bình -6.77%

Mã AFX: 63 lần mua bán, 

In [13]:
table = pd.DataFrame(columns=['Số lần', 'Trung bình', 'Lớn nhất', 'Nhỏ nhất'])
table.loc['Lãi'] = [len(total_profit), total_profit.mean(), total_profit.max(), total_profit.min()]
table.loc['Lỗ'] = [len(total_loss), total_loss.mean(), total_loss.min(), total_loss.max()]
table.loc['Hòa vốn'] = [len(total_breakeven), 0, 0, 0]
table.loc['Tổng kết'] = [len(total_result), total_result.mean(), total_result.max(), total_result.min()]

In [14]:
table

,Số lần,Trung bình,Lớn nhất,Nhỏ nhất
Lãi,43362.0,8.373816,1710.344828,1.118613e-14
Lỗ,28707.0,-5.764265,-100.000000,-1.117206e-14
Hòa vốn,5725.0,0.000000,0.000000,0.000000e+00
Tổng kết,77794.0,2.540436,1710.344828,-1.000000e+02
